# 开始使用

本文介绍如何使用 cedarkit 工具栈为 CMA-MESO 数据绘制中国区域 2 米温度填充图。

首先加载需要使用的包，包括：

- 数据结构相关：pandas
- 数据准备工具库：reki
- 绘图工具库：cedarkit.maps
    - 绘图板：`Panel`
    - 底图布局：`EastAsiaMapDomain`
    - 填充图样式：`ContourStyle`
    - 颜色表：`get_ncl_colormap`
    - 地图包：`set_default_map_package`

In [ ]:
import pandas as pd

from reki.data_finder import find_local_file
from reki.format.grib.eccodes import load_field_from_file

from cedarkit.maps.style import ContourStyle
from cedarkit.maps.chart import Panel
from cedarkit.maps.domains import EastAsiaMapDomain
from cedarkit.maps.colormap import get_ncl_colormap
from cedarkit.maps.map import set_default_map_package

设置绘图的数据参数。绘制 CMA-MESO 2024 年 4 月 1 日 00 时次 024 时效的 2 米温度场。

In [ ]:
system_name = "CMA-MESO"
data_type = "cma_meso_3km/grib2/orig"
start_time = pd.to_datetime("2024-04-01 00:00:00")
forecast_time = pd.to_timedelta("24h")

## 加载数据

获取 GRIB2 文件目录

In [ ]:
file_path = find_local_file(
    data_type,
    start_time=start_time,
    forecast_time=forecast_time,
)
file_path

加载 2 米温度场。这里直接使用 ecCodes 内置的要素名 `2t`

In [ ]:
t_2m_field = load_field_from_file(
    file_path,
    parameter="2t",
) - 273.15
t_2m_field

## 配置

定义填充图层次

In [ ]:
t_2m_level = [-24, -20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20, 24, 28, 32]

定义填充图颜色表。色表来自 cedarkit-maps 内置的 NCL 色表文件 BlAqGrYeOrReVi200.rgb。

In [ ]:
color_index = [2, 12, 22, 32, 42, 52, 62, 72, 82, 92, 102, 112, 122, 132, 142, 152]
t_2m_color_map = get_ncl_colormap("BlAqGrYeOrReVi200", index=color_index)

定义填充图样式

In [ ]:
t_2m_style = ContourStyle(
    colors=t_2m_color_map,
    levels=t_2m_level,
    fill=True,
)

使用 CEMC 地图包

> 注：如果没有安装 cemc-meda-data 包，可以注释改行代码，直接使用默认地图包

In [ ]:
set_default_map_package("cedarkit.maps.map.cemc")

## 绘图

创建中国区域底图布局

In [ ]:
domain = EastAsiaMapDomain()

创建绘图板，绘制等值线，设置标题，添加颜色条

In [ ]:
panel = Panel(domain=domain)
panel.plot(t_2m_field, style=t_2m_style)
domain.set_title(
    panel=panel,
    graph_name="2m Temperature (C)",
    system_name=system_name,
    start_time=start_time,
    forecast_time=forecast_time,
)
domain.add_colorbar(panel=panel, style=t_2m_style)
panel.show()